In [1]:
import numpy as np
import speaker_recog as spr
import os
import python_speech_features
import tensorflow as tf
from tensorflow import keras
import sounddevice as sd
from timeit import default_timer as timer
from IPython.display import clear_output
import speech_recognition as sr

In [2]:
def audio2feature(audio):
    features = python_speech_features.base.mfcc( audio, 
                                                 samplerate = 16000,
                                                 winlen = 0.025,
                                                 winstep = 0.01,
                                                 numcep = 20,
                                                 nfilt = 20,
                                                 nfft = 2048,
                                                 preemph = 0,
                                                 ceplifter = 0,
                                                 appendEnergy = False,
                                                 winfunc = np.hanning)
    return features.transpose()

In [3]:
model = keras.models.load_model('cmd_recog_model.h5')

In [4]:
dataset_path = './dataset'
targets = [name for name in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, name))]
#targets = ['on', 'two', 'one', 'three', 'marvin', 'off']
print(targets)

['on', 'two', 'one', 'three', 'off']


In [6]:
def predict_cmd():
    print("Listening for command...")
    cmd_rec = sd.rec(samplerate=16000, 
                       channels=1,
                       dtype="float32",
                       frames=16000 * 1)
    sd.wait()
    print("Recording stopped")
    recorded_feature = audio2feature(cmd_rec)
    recorded_feature = np.float32(recorded_feature.reshape(1, recorded_feature.shape[0], recorded_feature.shape[1], 1))
    prediction = model.predict(recorded_feature).reshape((5, ))
    prediction /= prediction.sum()
    best_candidate_index = prediction.argmax()
    best_candidate_probability = prediction[best_candidate_index]
    word = targets[best_candidate_index]
    speak_verif = spr.test_model(cmd_rec, 16000)
    if speak_verif == True:
        return word
    else:
        return None

In [7]:
r = sr.Recognizer()
mic = sr.Microphone()
def speech_to_text():
    while True:
        with mic as source:
            r.adjust_for_ambient_noise(source)
            audio = r.listen(source)
        audioText = r.recognize_google(audio)
        if audioText == "motor":
            print(audioText)
            cmd = predict_cmd()
            if cmd != None:
                print("Recieved command : ", cmd)
            else:
                print("Invalid User.")
        else:
            print("Wakeword not detected!")
            
def main_func():
    try:
        speech_to_text()
    except Exception as e:
        print(str(e))
        print("Sry,we couldnt get your voice")
        print("Try again")
        main_func()

if __name__ == "__main__":
    main_func()


Sry,we couldnt get your voice
Try again

Sry,we couldnt get your voice
Try again
motor
Listening for command...
Recording stopped
[GaussianMixture(covariance_type='diag', max_iter=200, n_components=20, n_init=3), GaussianMixture(covariance_type='diag', max_iter=200, n_components=20, n_init=3), GaussianMixture(covariance_type='diag', max_iter=200, n_components=20, n_init=3)]
[-25.78586908 -25.964852   -26.58792426]
	detected as -  ./trained_models/gopi
Recieved command :  off
Moto
Moto
motor
Listening for command...
Recording stopped
[GaussianMixture(covariance_type='diag', max_iter=200, n_components=20, n_init=3), GaussianMixture(covariance_type='diag', max_iter=200, n_components=20, n_init=3), GaussianMixture(covariance_type='diag', max_iter=200, n_components=20, n_init=3)]
[-26.11051375 -25.75716457 -26.61097155]
	detected as -  ./trained_models/anirudh
Recieved command :  off

Sry,we couldnt get your voice
Try again

Sry,we couldnt get your voice
Try again
motor
Listening for comma

KeyboardInterrupt: 